In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, cross_validate, validation_curve
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score
warnings.simplefilter(action='ignore', category=Warning)
from sklearn.model_selection import learning_curve


Bad key "text.kerning_factor" on line 4 in
C:\Users\DK\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
df=pd.read_csv("ionosphere.csv",sep=",")
df.dropna(inplace=True)
df.head()

,a01,a02,a03,a04,a05,a06,a07,a08,a09,a10,...,a26,a27,a28,a29,a30,a31,a32,a33,a34,class
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [3]:
# converting from string to numeric
df["class"]=df["class"].map({"g":1,"b":0})

In [4]:
y = df["class"] # dependent(target) variable
X = df.drop(["class"], axis=1) # independent variables

In [5]:
gbm_model = GradientBoostingClassifier(n_estimators = 50)

In [6]:
model = gbm_model.fit(X,y)

In [8]:
cv_results = cross_validate(gbm_model, X, y, cv=10, scoring=["f1"],
                            return_train_score=True)

print("trainscore:",cv_results['train_f1'].mean())
print("test score:",cv_results['test_f1'].mean())

trainscore: 0.9968034463605795
test score: 0.9428214008017853


In [9]:
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier

In [11]:
ada_model = AdaBoostClassifier(n_estimators=50)

In [12]:
model = ada_model.fit(X,y)

In [13]:
cv_results = cross_validate(ada_model, X, y, cv=10, scoring=["f1"],
                            return_train_score=True)

print("trainscore:",cv_results['train_f1'].mean())
print("test score:",cv_results['test_f1'].mean())

trainscore: 1.0
test score: 0.9403028396551774


In [16]:
from sklearn.ensemble import StackingClassifier
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [17]:
# Add Logistic Regression model to estimators

estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))

# Add Decision Tree model to estimators
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))

# Add Support Vector Machine model to estimators
model3 = SVC()
estimators.append(('svm', model3))

In [18]:
SC = StackingClassifier(estimators=estimators,final_estimator=gbm_model,cv=6)

In [19]:
SC.fit(X,y)

StackingClassifier(cv=6,
                   estimators=[('logistic', LogisticRegression()),
                               ('cart', DecisionTreeClassifier()),
                               ('svm', SVC())],
                   final_estimator=GradientBoostingClassifier(n_estimators=50))

In [20]:
cv_results = cross_validate(SC, X, y, cv=10, scoring=["f1"],
                            return_train_score=True)

print("trainscore:",cv_results['train_f1'].mean())
print("test score:",cv_results['test_f1'].mean())

trainscore: 0.9695884084839014
test score: 0.9386164475901584
